<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>LangChain for Generative AI</h1>
<h1>ChatBot</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [ ]:
!pip install -U sentence-transformers

In [22]:
!pip install -U chromadb

  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached chroma_hnswlib-0.7.3-cp312-cp312-macosx_10_9_universal2.whl
  Using cached posthog-3.5.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached kubernetes-29.0.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached mmh3-4.1.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached google_auth-2.29.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 by

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 2.0 MB/s eta 0:00:00MB/s eta 0:00:01


In [18]:
from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import langchain

from langchain.prompts import PromptTemplate
from langchain.document_loaders import GutenbergLoader

from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories.in_memory import ChatMessageHistory

from langchain.schema import messages_from_dict, messages_to_dict

from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from langchain.chains import LLMChain, ConversationalRetrievalChain, ConversationChain
from langchain.chains import RetrievalQA

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

import langchain_openai
from langchain_openai import ChatOpenAI

import tempfile

import watermark

import sentence_transformers

%load_ext watermark
%matplotlib inline

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


We start by print out the versions of the libraries we're using for future reference

In [10]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.12.1
IPython version      : 8.21.0

Compiler    : Clang 13.0.0 (clang-1300.0.29.30)
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

Git hash: cc526109102c100be511b85d0872c4bdfd674d94

langchain_openai: 0.1.3
watermark       : 2.4.3
numpy           : 1.26.4
matplotlib      : 3.8.3
pandas          : 2.2.1
langchain       : 0.1.16



Load default figure style

In [11]:
plt.style.use('./d4sci.mplstyle')

# Start

In [12]:
cache_dir = "./cache"

In [23]:

loader = GutenbergLoader(
    "https://www.gutenberg.org/cache/epub/1513/pg1513.txt"
)

document = loader.load()

extrait = ' '.join(document[0].page_content.split()[:100])
display(extrait + " .......")

'The Project Gutenberg eBook of Romeo and Juliet This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook. Title: Romeo and Juliet Author: William Shakespeare Release date: November .......'

In [24]:
text_splitter = CharacterTextSplitter(
    chunk_size=1024, # Each chunk is of size 1024
    chunk_overlap=256 # Neigboring chunks overlap by 256 characters
) 

texts = text_splitter.split_documents(document)

In [25]:
print(len(texts))

233


In [26]:
# Storing in a Vector DB 

model_name = "sentence-transformers/all-MiniLM-L6-v2"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name, 
    cache_folder=cache_dir
)  # Use a pre-cached model

vectordb = Chroma.from_documents(
    texts, 
    embeddings, 
    persist_directory=cache_dir
)


In [27]:
question = "Romeo!"

docs = vectordb.similarity_search(question, k=2)

# Check the length of the document
print(len(docs))

# Check the content of the first document
print(docs[0].page_content)

2
That same villain Romeo.





JULIET.


Villain and he be many miles asunder.


God pardon him. I do, with all my heart.


And yet no man like he doth grieve my heart.





LADY CAPULET.


That is because the traitor murderer lives.





JULIET.


Ay madam, from the reach of these my hands.


Would none but I might venge my cousin’s death.





LADY CAPULET.


We will have vengeance for it, fear thou not.


Then weep no more. I’ll send to one in Mantua,


Where that same banish’d runagate doth live,


Shall give him such an unaccustom’d dram


That he shall soon keep Tybalt company:


And then I hope thou wilt be satisfied.





JULIET.


Indeed I never shall be satisfied


With Romeo till I behold him—dead—


Is my poor heart so for a kinsman vex’d.


Madam, if you could find out but a man


To bear a poison, I would temper it,


That Romeo should upon receipt thereof,


Soon sleep in quiet. O, how my heart abhors


To hear him nam’d, and cannot come to him,


Create a wrapper around the functionality of our vector database so we can search for similar documents in the vectorstore

In [9]:
retriever = vectordb.as_retriever()

In [10]:
llm = ChatOpenAI(model='gpt-3.5-turbo',temperature=0)

In [11]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [12]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [13]:
query = "What is Juliets family?"
query_results_venice = qa.invoke(query)
print("#" * 12)
query_results_venice['result']

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


############


'The Capulets. Thanks for asking!'

In [14]:
query = "What happens to Romeo and Juliet?"
query_results_romeo = qa.invoke(query)
print("#" * 12)
query_results_romeo['result']

############


'Romeo and Juliet both die by the end of the play. Thanks for asking!'

In [15]:
query = "Who is Mercutio?"
query_results_romeo = qa.invoke(query)
print("#" * 12)
query_results_romeo['result']

############


'Mercutio is a character in William Shakespeare\'s play "Romeo and Juliet." He is a close friend of Romeo and is known for his wit and humor. Thanks for asking!'

In [16]:
query = "Does Romeo live?"
qa_chain_docs = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       # Return source documents
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain_docs({"query": question})
result["result"]

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


"I don't know.\nThanks for asking!"

In [17]:
len(result['source_documents'])

4

In [18]:
print(result['source_documents'][0].page_content)

That same villain Romeo.





JULIET.


Villain and he be many miles asunder.


God pardon him. I do, with all my heart.


And yet no man like he doth grieve my heart.





LADY CAPULET.


That is because the traitor murderer lives.





JULIET.


Ay madam, from the reach of these my hands.


Would none but I might venge my cousin’s death.





LADY CAPULET.


We will have vengeance for it, fear thou not.


Then weep no more. I’ll send to one in Mantua,


Where that same banish’d runagate doth live,


Shall give him such an unaccustom’d dram


That he shall soon keep Tybalt company:


And then I hope thou wilt be satisfied.





JULIET.


Indeed I never shall be satisfied


With Romeo till I behold him—dead—


Is my poor heart so for a kinsman vex’d.


Madam, if you could find out but a man


To bear a poison, I would temper it,


That Romeo should upon receipt thereof,


Soon sleep in quiet. O, how my heart abhors


To hear him nam’d, and cannot come to him,


<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>